In [2]:
import tensorflow as tf
import os
import pickle
import numpy as np

CIFAR_DIR = "../04-advanced-conv/cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

['data_batch_1', 'data_batch_5', 'data_batch_2', 'test_batch', 'batches.meta', 'readme.html', 'data_batch_4', 'data_batch_3']


In [3]:
# tensorboard
# 1. 指定面板图上显示的变量
# 2. 训练过程中将这些变量计算出来，输出到文件中
# 3. 文件解析 ./tensorboard --logdir=dir.

# 以下优化都在计算图的构建环节
# activation: relu, sigmoid, tanh
# weight initializer: he（何开明发明的）, xavier, normal（正态分布）, truncated_normal
# truncated_normal函数产生正太分布，均值和标准差自己设定
# optimzier: Adam, Momentum, Gradient Descent.

In [4]:
def load_data(filename):
    """read data from data file."""
    with open(filename, 'rb') as f:
        data = pickle.load(f, encoding='bytes')
        return data[b'data'], data[b'labels']


class CifarData:
    def __init__(self, filenames, need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data, labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
            
    def _shuffle_data(self):
        # [0,1,2,3,4,5] -> [5,3,2,4,0,1]
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]
    
    def next_batch(self, batch_size):
        """return batch_size examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator: end_indicator]
        batch_labels = self._labels[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i) for i in range(1, 6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames, True)
test_data = CifarData(test_filenames, False)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [5]:
x = tf.placeholder(tf.float32, [None, 3072])
y = tf.placeholder(tf.int64, [None])
# [None], eg: [0,5,6,3]
x_image = tf.reshape(x, [-1, 3, 32, 32])
# 32*32
x_image = tf.transpose(x_image, perm=[0, 2, 3, 1])

# 这里相对于之前代码进行了封装，把activation，kernel_initializer（初始化参数）变为参数，把卷积层和池化层都封装到一个函数里
def convnet(inputs, activation, kernel_initializer):
    # conv1: 神经元图， feature_map, 输出图像
    conv1_1 = tf.layers.conv2d(inputs,
                               32,  # output channel number
                               (3,3),  # kernel size
                               padding = 'same',
                               activation = activation,
                               kernel_initializer = kernel_initializer,
                               name = 'conv1_1')
    conv1_2 = tf.layers.conv2d(conv1_1,
                               32,  # output channel number
                               (3,3),  # kernel size
                               padding = 'same',
                               activation = activation,
                               kernel_initializer = kernel_initializer,
                               name = 'conv1_2')

    # 16 * 16
    pooling1 = tf.layers.max_pooling2d(conv1_2,
                                       (2, 2),  # kernel size
                                       (2, 2),  # stride
                                       name = 'pool1')


    conv2_1 = tf.layers.conv2d(pooling1,
                               32,  # output channel number
                               (3,3),  # kernel size
                               padding = 'same',
                               activation = activation,
                               kernel_initializer = kernel_initializer,
                               name = 'conv2_1')
    conv2_2 = tf.layers.conv2d(conv2_1,
                               32,  # output channel number
                               (3,3),  # kernel size
                               padding = 'same',
                               activation = activation,
                               kernel_initializer = kernel_initializer,
                               name = 'conv2_2')
    # 8 * 8
    pooling2 = tf.layers.max_pooling2d(conv2_2,
                                       (2, 2),  # kernel size
                                       (2, 2),  # stride
                                       name = 'pool2')

    conv3_1 = tf.layers.conv2d(pooling2,
                               32,  # output channel number
                               (3,3),  # kernel size
                               padding = 'same',
                               activation = activation,
                               kernel_initializer = kernel_initializer,
                               name = 'conv3_1')
    conv3_2 = tf.layers.conv2d(conv3_1,
                               32,  # output channel number
                               (3,3),  # kernel size
                               padding = 'same',
                               activation = activation,
                               kernel_initializer = kernel_initializer,
                               name = 'conv3_2')
    # 4 * 4 * 32
    pooling3 = tf.layers.max_pooling2d(conv3_2,
                                       (2, 2),  # kernel size
                                       (2, 2),  # stride
                                       name = 'pool3')
    # [None, 4 * 4 * 32]
    flatten = tf.layers.flatten(pooling3)
    return flatten

# 备注是测试效果
# sigmoid: 53.39% vs relu: 73.35% on 10k train  sigmoid会比relu慢很多
# tf.glorot_uniform_initializer: 76.53% 100k train.  glorot_uniform_initializer是默认的初始化方法
# tf.truncated_normal_initializer: 78.04% 100k train   truncated_normal_initializer随机产生的值在一定范围内
# tf.keras.initializers.he_normal: 71.52% 100k train  he的参数没有表现比较好，可能是因为我们的网络并不是一个很深层次的网络
flatten = convnet(x_image, tf.nn.relu, tf.truncated_normal_initializer(stddev=0.02))

y_ = tf.layers.dense(flatten, 10)

loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> sofmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_, 1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
    # gradient descent: 12.35% train 100k  普通的梯度下降
    # train_op = tf.train.GradientDescentOptimizer(1e-4).minimize(loss)
    # momentum: 35.75% train 100k.比较低
    # reason: 1. initializer incorrect. 2. 不充分的训练  momentum是冲量
    # momentum表示要在多大程度上保留原来的更新方向，这个值在0-1之间，在训练开始时，由于梯度可能会很大，所以初始值一般选为0.5；当梯度不那么大时，改为0.9
    # train_op = tf.train.MomentumOptimizer(learning_rate=1e-4, momentum=0.9).minimize(loss)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.


In [6]:
def variable_summary(var, name):
    """Constructs summary for statistics of a variable"""
    with tf.name_scope(name):
        mean = tf.reduce_mean(var)
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('mean', mean)
        tf.summary.scalar('stddev', stddev)
        tf.summary.scalar('min', tf.reduce_min(var))
        tf.summary.scalar('max', tf.reduce_max(var))
        tf.summary.histogram('histogram', var)

"""
with tf.name_scope('summary'):
    variable_summary(conv1_1, 'conv1_1')
    variable_summary(conv1_2, 'conv1_2')
    variable_summary(conv2_1, 'conv2_1')
    variable_summary(conv2_2, 'conv2_2')
    variable_summary(conv3_1, 'conv3_1')
    variable_summary(conv3_2, 'conv3_2')
"""

loss_summary = tf.summary.scalar('loss', loss)
# 'loss': <10, 1.1>, <20, 1.08>
accuracy_summary = tf.summary.scalar('accuracy', accuracy)

source_image = (x_image + 1) * 127.5
inputs_summary = tf.summary.image('inputs_image', source_image)

merged_summary = tf.summary.merge_all()
merged_summary_test = tf.summary.merge([loss_summary, accuracy_summary])

LOG_DIR = '.'
run_label = 'run_vgg_tensorboard_activation_initializer_optimizer'
run_dir = os.path.join(LOG_DIR, run_label)
if not os.path.exists(run_dir):
    os.mkdir(run_dir)
train_log_dir = os.path.join(run_dir, 'train')
test_log_dir = os.path.join(run_dir, 'test')
if not os.path.exists(train_log_dir):
    os.mkdir(train_log_dir)
if not os.path.exists(test_log_dir):
    os.mkdir(test_log_dir)

In [7]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 10000
test_steps = 100

output_summary_every_steps = 100


# train 10000: 70.45%
with tf.Session() as sess:
    sess.run(init)
    train_writer = tf.summary.FileWriter(train_log_dir, sess.graph)
    test_writer = tf.summary.FileWriter(test_log_dir)
    
    fixed_test_batch_data, fixed_test_batch_labels \
        = test_data.next_batch(batch_size)

    for i in range(train_steps):
        batch_data, batch_labels = train_data.next_batch(batch_size)
        eval_ops = [loss, accuracy, train_op]
        should_output_summary = ((i+1) % output_summary_every_steps == 0)
        if should_output_summary:
            eval_ops.append(merged_summary)
    
        eval_ops_results = sess.run(
            eval_ops,
            feed_dict={
                x: batch_data,
                y: batch_labels})
        loss_val, acc_val = eval_ops_results[0:2]
        if should_output_summary:
            train_summary_str = eval_ops_results[-1]
            train_writer.add_summary(train_summary_str, i+1)
            test_summary_str = sess.run([merged_summary_test],
                                        feed_dict={
                                            x: fixed_test_batch_data,
                                            y: fixed_test_batch_labels,
                                        })[0]
            test_writer.add_summary(test_summary_str, i+1)
    
        if (i+1) % 100 == 0:
            print('[Train] Step: %d, loss: %4.5f, acc: %4.5f' % (i+1, loss_val, acc_val))
        if (i+1) % 1000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data, test_batch_labels \
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuracy],
                    feed_dict = {
                        x: test_batch_data, 
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.mean(all_test_acc_val)
            print('[Test ] Step: %d, acc: %4.5f' % (i+1, test_acc))

[Train] Step: 100, loss: 2.29329, acc: 0.05000
[Train] Step: 200, loss: 1.86712, acc: 0.15000
[Train] Step: 300, loss: 1.71569, acc: 0.40000
[Train] Step: 400, loss: 1.82255, acc: 0.35000
[Train] Step: 500, loss: 1.90590, acc: 0.20000
[Train] Step: 600, loss: 2.12399, acc: 0.30000
[Train] Step: 700, loss: 1.90179, acc: 0.25000
[Train] Step: 800, loss: 1.59777, acc: 0.40000
[Train] Step: 900, loss: 1.55929, acc: 0.45000
[Train] Step: 1000, loss: 1.66240, acc: 0.35000
(10000, 3072)
(10000,)
[Test ] Step: 1000, acc: 0.40050
[Train] Step: 1100, loss: 1.78558, acc: 0.45000
[Train] Step: 1200, loss: 1.45564, acc: 0.55000
[Train] Step: 1300, loss: 1.45945, acc: 0.45000
[Train] Step: 1400, loss: 1.72523, acc: 0.35000
[Train] Step: 1500, loss: 1.64026, acc: 0.45000
[Train] Step: 1600, loss: 1.31405, acc: 0.45000
[Train] Step: 1700, loss: 1.38344, acc: 0.40000
[Train] Step: 1800, loss: 1.61994, acc: 0.55000
[Train] Step: 1900, loss: 1.62620, acc: 0.40000
[Train] Step: 2000, loss: 1.03406, acc: 0